## TRABALHO DE PYTHON - ANÁLISE DE DADOS - COSMETIC SHOP

---------------------------------------------------------------------------
Ícaro Gabriel  
Eduardo Farias  
Pedro

---------------------------------------------------------------------------
### ***INFORMAÇÕES***
Esse banco de dados possui uma vasta lista de eventos que aconteceram dentro de 5 meses em uma loja de cosmético. Cada linha representa um evento. Todos os eventos são relacionados a produtos e usuários.

_"Usuário **user_id** durante uma sessão **user_session** adicionou ao carrinho (se **event_type == cart**) o produto **product_id** da marca **brand** da categoria **category_code** com o preço **price** no momento **event_time**."_

***ESTRUTURA***
| Propriedade | Descrição |
|-------------|-----------|
| event_time | Data que o evento aconteceu |
| event_type | Tipo de evento (tipos listados abaixo) |
| product_id | ID do produto relacionado ao evento |
| category_id | ID da categoria do product_id |
| category_code | Taxonomia/nome da categoria do produto, se é possível definir. Geralmente presente em categorias específicas |
| brand | String em letras minúsculas da marca do produto |
| price | Número float do preço do produto |
| user_id | ID do usuário |
| user_session | ID da sessão temporária do usuário. Muda toda vez que o usuário sai e entra novamente depois de um tempo. |  

***TIPOS DE EVENTO***                                    
|- **view**: o usuário visualizou o produto.  
|- **cart**: o usuário adicionou um produto ao carrinho.  
|- **remove_from_cart**: o usuário removeu um produto do carrinho.  
|- **purchase**: o usuário comprou um produto.  

***OBSERVAÇÃO***  
|- Uma única sessão pode ter vários eventos de purchase (compra). Nesse caso, seria considerado uma única ordem.


In [2]:
import pandas as pd 
import numpy as np

In [3]:
# Nome dos arquivos
files = ['2019-Oct.csv', '2019-Nov.csv', '2019-Dec.csv', '2020-Jan.csv', '2020-Feb.csv']

# Adicionando DataFrames a uma lista
dataframes = []
for file in files:
    dataframes.append(pd.read_csv(file))
    
# Concatenando todos os DataFrames
data = pd.concat(dataframes, ignore_index=True)

----------------------------------------------------
Abaixo, estão listados os cinco primeiros eventos. Utilizando eles como exemplo, mostrarei como ler corretamente:
- O primeiro dado que vemos é o index do evento dentro do DataFrame. No caso abaixo, vai de 0 a 4.
- Porém, podemos ver que temos 3 blocos diferentes.
- O index é único para cada, e o Python separou os dados em 3 blocos. Então o evento de index 0 tem seus outros dados também no index 0 nos outros blocos.

Exemplo:

| index | event_time | event_type | product_id | category_id | category_code | brand | price | user_id | user_session |
| ------|------------|------------|------------|-------------|---------------|-------|-------|---------|--------------|
|0 |2019-10-01 00:00:00 UTC | cart|5773203 |1487580005134238553 |NaN |runail |2.62 |463240011 |26dd6e6e-4dac-4778-8d2c-92e149dab885 |

In [24]:
print(data.head(50))

                  event_time        event_type  product_id  \
0  2019-10-01 00:00:00+00:00              cart     5773203   
1  2019-10-01 00:00:03+00:00              cart     5773353   
2  2019-10-01 00:00:07+00:00              cart     5881589   
3  2019-10-01 00:00:07+00:00              cart     5723490   
4  2019-10-01 00:00:15+00:00              cart     5881449   
5  2019-10-01 00:00:16+00:00              cart     5857269   
6  2019-10-01 00:00:19+00:00              cart     5739055   
7  2019-10-01 00:00:24+00:00              cart     5825598   
8  2019-10-01 00:00:25+00:00              cart     5698989   
9  2019-10-01 00:00:26+00:00              view     5875317   
10 2019-10-01 00:00:28+00:00              view     5692917   
11 2019-10-01 00:00:28+00:00  remove_from_cart     5834172   
12 2019-10-01 00:00:30+00:00  remove_from_cart     5809103   
13 2019-10-01 00:00:30+00:00  remove_from_cart     5809103   
14 2019-10-01 00:00:32+00:00  remove_from_cart     5779403   
15 2019-

In [37]:
quantity_lines = data.shape[0]
print(f"Quantidade de linhas no DataFrame (Antes da limpeza): {quantity_lines}")

Quantidade de linhas no DataFrame (Antes da limpeza): 20692840


----------------------------------------------------------
## LIMPEZA

In [7]:
# Tratando espaços vazios do category_code - Vazios serão preenchidos com 'undefined'
data['category_code'] = data['category_code'].fillna('undefined')
data['brand'] = data['brand'].fillna('undefined')

In [10]:
# Confirmando se os tipos de dados estão corretos (otimização)
data['event_time'] = pd.to_datetime(data['event_time'])
data['price'] = data['price'].astype('float32')
data['user_id'] = data['user_id'].astype('int32')
data['product_id'] = data['product_id'].astype('int32')
data['event_type'] = data['event_type'].astype('category')
data['category_code'] = data['category_code'].astype('category')
data['brand'] = data['brand'].astype('category')

In [25]:
# Verificar valores incorretos
incorrect_price = data[data['price'] <= 0]
print(f"Número de linhas com 'price' incorreto (<= 0): {incorrect_price.shape[0]}")
print(incorrect_price.head(10))


Número de linhas com 'price' incorreto (<= 0): 104288
                     event_time event_type  product_id          category_id  \
2454  2019-10-01 02:05:17+00:00       view     5892084  1597770225539875791   
2614  2019-10-01 02:15:41+00:00       view     5892052  1487580010377117763   
5686  2019-10-01 04:25:00+00:00       view     5873432  2007399943458784057   
6086  2019-10-01 04:33:15+00:00       view     5882605  1487580013522845895   
8344  2019-10-01 05:16:30+00:00       view     5889621  1487580010561667147   
8417  2019-10-01 05:18:03+00:00       view     5889622  1487580010561667147   
8465  2019-10-01 05:18:46+00:00       view     5889623  1487580010561667147   
8950  2019-10-01 05:24:38+00:00       view     5873431  2007399943458784057   
10125 2019-10-01 05:38:01+00:00       view     5889627  1487580010561667147   
10154 2019-10-01 05:38:32+00:00       view     5889628  1487580010561667147   

       category_code      brand  price    user_id  \
2454       undefined  u


- Sobre: Como é possível ver, em muitas colunas o preço está inconsistente. Portanto, deve-se analisar para verificar se é um erro.        








In [28]:
# Verificar se em outros eventos o mesmo produto com price errado está errado também
incorrect_price_products = data[data['price'] <= 0]['product_id'].unique()

related_events = data[data['product_id'].isin(incorrect_price_products)]

related_events = related_events.sort_values(by='product_id')

print(related_events.head(50))

                        event_time event_type  product_id  \
2637600  2019-10-20 13:50:41+00:00       view        3763   
2872640  2019-10-22 08:45:58+00:00       view        3763   
635194   2019-10-05 14:20:55+00:00       cart        3763   
2697827  2019-10-20 23:15:50+00:00       view        3763   
10435198 2019-12-13 18:41:39+00:00       view        3763   
2537290  2019-10-19 15:05:42+00:00       view        3763   
7906441  2019-11-26 10:10:18+00:00       view        3763   
4519556  2019-11-04 12:57:49+00:00       view        3763   
40458    2019-10-01 10:04:29+00:00       view        3763   
40481    2019-10-01 10:04:43+00:00       cart        3763   
11485978 2019-12-22 18:41:39+00:00       cart        3763   
2460863  2019-10-18 19:09:50+00:00       view        3763   
5777330  2019-11-13 07:16:43+00:00       view        3763   
2300460  2019-10-17 13:23:36+00:00       view        3763   
41037    2019-10-01 10:09:01+00:00       view        3763   
2537552  2019-10-19 15:0

- Sobre: Pegando de exemplo o product_id 3763, que aparece acima, podemos ver que seu preço fica por volta de *16.030001*








In [33]:
# Verificar os mesmos produtos incorretos, agora filtrando para ver os com valores 0 ou menos. O objetivo é verificar se o mesmo id 3763 aparece com um valor errado.

related_events = data[(data['product_id'].isin(incorrect_price_products)) & (data['price'] < 0)]

related_events = related_events.sort_values(by='product_id')

print(related_events.head(50))


                        event_time event_type  product_id  \
443204   2019-10-03 18:25:39+00:00   purchase     5670257   
3593034  2019-10-28 04:19:20+00:00   purchase     5670257   
2286569  2019-10-17 11:41:07+00:00   purchase     5670257   
2418566  2019-10-18 12:27:23+00:00   purchase     5670257   
3056821  2019-10-23 15:11:37+00:00   purchase     5670257   
4852226  2019-11-06 17:31:51+00:00   purchase     5670257   
4510519  2019-11-04 11:54:29+00:00   purchase     5670257   
3670216  2019-10-28 15:43:59+00:00   purchase     5670257   
4809167  2019-11-06 11:51:26+00:00   purchase     5670257   
11958493 2019-12-27 08:34:06+00:00   purchase     5670257   
12034002 2019-12-28 07:39:08+00:00   purchase     5670257   
10902771 2019-12-17 13:31:01+00:00   purchase     5670257   
10881821 2019-12-17 10:37:03+00:00   purchase     5670257   
9680169  2019-12-08 12:38:35+00:00   purchase     5670257   
9431181  2019-12-06 08:44:06+00:00   purchase     5670257   
8442913  2019-11-29 10:4

- Sobre: Como é possível ver, as células do 3763 aparecem aqui com valores diferentes da média comum, que era por volta de 16.0. Agora, eles aparecem zerados.
- Não é possível ter um produto com valor 0 ou negativo, pois isso seria dar de graça ou dar dinheiro ao cliente, respectivamente.
- Portanto, como o objetivo é a análise do comportamento dos clientes, devemos apagar esses dados inconsistentes.

In [38]:
# Apagar dados com valores errados
data_cleaned = data[data['price'] > 0]
print(data_cleaned['price'].describe())
print(f"Quantidade de linhas do DataFrame: {data_cleaned.shape[0]}")

count    2.058855e+07
mean     8.578157e+00
std      1.942060e+01
min      5.000000e-02
25%      2.060000e+00
50%      4.110000e+00
75%      7.140000e+00
max      3.277800e+02
Name: price, dtype: float64
Quantidade de linhas do DataFrame: 20588552


In [54]:
# Verificar se o event_type tem os tipos corretos em todas as linhas
print(f"Valores únicos em event_type: {data_cleaned['event_type'].unique()}")

Valores únicos em event_type: ['cart', 'view', 'remove_from_cart', 'purchase']
Categories (4, object): ['cart', 'purchase', 'remove_from_cart', 'view']


In [55]:
# Verificar datas
date_min = data_cleaned['event_time'].min()
date_max = data_cleaned['event_time'].max()

print(f"Data mínima: {date_min}")
print(f"Data máxima: {date_max}")

Data mínima: 2019-10-01 00:00:00+00:00
Data máxima: 2020-02-29 23:59:59+00:00


--------------------------------------
## TABELAS

### Tabela de Produtos

In [56]:
product_data = data_cleaned[['product_id', 'price', 'category_id', 'category_code']]
product_data = product_data.drop_duplicates(subset='product_id')
print(f"Número de produtos: {product_data.shape[0]}")
print(product_data.head(50))

Número de produtos: 53904
    product_id      price          category_id                  category_code
0      5773203   2.620000  1487580005134238553                      undefined
1      5773353   2.620000  1487580005134238553                      undefined
2      5881589  13.480000  2151191071051219817                      undefined
3      5723490   2.620000  1487580005134238553                      undefined
4      5881449   0.560000  1487580013522845895                      undefined
5      5857269   2.620000  1487580005134238553                      undefined
6      5739055   4.750000  1487580008246412266                      undefined
7      5825598   0.560000  1487580009445982239                      undefined
8      5698989   1.270000  1487580006317032337                      undefined
9      5875317   1.590000  2029082628195353599                      undefined
10     5692917   5.540000  1487580004857414477                      undefined
11     5834172   0.950000  14875800135

### Tabela de Categorias com category_code válidos

In [57]:
category_data = data_cleaned[['category_id', 'category_code']]
category_data = category_data[category_data['category_code'] != 'undefined']
category_data = category_data.drop_duplicates(subset='category_id')
print(f"Número de categorias: {category_data.shape[0]}")
print(category_data.head(20))

Número de categorias: 18
                  category_id                           category_code
21        1487580006350586771           appliances.environment.vacuum
264       1487580013053083824                     stationery.cartrige
877       2007399943458784057                           apparel.glove
1723      1487580010695884882                         accessories.bag
1928      2193074740619379535           furniture.living_room.cabinet
2940      2193074740686488401                 furniture.bathroom.bath
4004      1487580008070251489         appliances.personal.hair_cutter
5204      1487580011970953351                 furniture.bathroom.bath
16998     1921723506584715388                accessories.cosmetic_bag
17877     1487580008221246441  appliances.environment.air_conditioner
39687     2022622168218599898             furniture.living_room.chair
41807     2018287324474901238                 furniture.bathroom.bath
51770     1487580012759482531                 furniture.bathroom.

### Tabela de Marcas

In [58]:
brand_data = data_cleaned[['brand']]
brand_data = brand_data.drop_duplicates(subset='brand')
print(f"Número de marcas: {brand_data.shape[0]}")
print(brand_data.head(50))

Número de marcas: 274
           brand
0         runail
2         lovely
6         kapous
7      undefined
10       lianail
12         irisk
17      nagaraku
20        masura
25     bpw.style
26         estel
28      ingarden
29        de.lux
37          milv
44         f.o.x
52           cnd
54       grattol
72       italwax
73       bluesky
83          pole
88          keen
95      jessnail
122      concept
123        dizao
132         yoko
143      bioaqua
153     haruyama
162         rosi
165        thuya
170     airnails
176     roubloff
184      beautix
187          uno
276    depilflax
281       entity
284    parachute
311          max
315        smart
330  beauty-free
337       strong
340      staleks
347   art-visage
356    freedecor
362       cutrin
369     severina
410     kinetics
436         shik
504          jas
517      metzger
545   cosmoprofi
550       matrix


### Tabela Geral com Informações Extras (Dia da Semana, Mês e Hora do Evento)

In [62]:
data_cleaned.loc[:, 'day_of_week'] = data_cleaned['event_time'].dt.day_name()
data_cleaned.loc[:, 'month'] = data_cleaned['event_time'].dt.month_name()
data_cleaned.loc[:, 'hour'] = data_cleaned['event_time'].dt.hour
print(data_cleaned.head(50))

                  event_time        event_type  product_id  \
0  2019-10-01 00:00:00+00:00              cart     5773203   
1  2019-10-01 00:00:03+00:00              cart     5773353   
2  2019-10-01 00:00:07+00:00              cart     5881589   
3  2019-10-01 00:00:07+00:00              cart     5723490   
4  2019-10-01 00:00:15+00:00              cart     5881449   
5  2019-10-01 00:00:16+00:00              cart     5857269   
6  2019-10-01 00:00:19+00:00              cart     5739055   
7  2019-10-01 00:00:24+00:00              cart     5825598   
8  2019-10-01 00:00:25+00:00              cart     5698989   
9  2019-10-01 00:00:26+00:00              view     5875317   
10 2019-10-01 00:00:28+00:00              view     5692917   
11 2019-10-01 00:00:28+00:00  remove_from_cart     5834172   
12 2019-10-01 00:00:30+00:00  remove_from_cart     5809103   
13 2019-10-01 00:00:30+00:00  remove_from_cart     5809103   
14 2019-10-01 00:00:32+00:00  remove_from_cart     5779403   
15 2019-

----------------------------------------------------------
## OUTRAS INFORMAÇÕES ÚTEIS

### Estatísticas Gerais

In [81]:
print(f"Quantidade de eventos: {data_cleaned.shape[0]}")
print(f"Média de preço geral: {data_cleaned['price'].mean()}")
print(f"Preço máximo: {data_cleaned['price'].max()}")
print(f"Preço mínimo: {data_cleaned['price'].min()}")
print("----------------------------------------------")
print(f"Quantidade de eventos por tipo: ")
print(data_cleaned['event_type'].value_counts())
print("----------------------------------------------")

Quantidade de eventos: 20588552
Média de preço geral: 8.578157424926758
Preço máximo: 327.7799987792969
Preço mínimo: 0.05000000074505806
----------------------------------------------
Quantidade de eventos por tipo: 
event_type
view                9627329
cart                5718963
remove_from_cart    3955380
purchase            1286880
Name: count, dtype: int64
----------------------------------------------


### Estatísticas por Data

In [80]:
print("----------------------------------------------")
print("Quantidade de eventos por dia da semana: ")
print(data_cleaned['day_of_week'].value_counts())
print("Legenda -> De cima para baixo: Quinta-feira, Quarta-feira, Terça-feira, Segunda-feira, Sexta-feira, Domingo e Sábado")
print("----------------------------------------------")
print("Quantidade de eventos por hora do dia: ")
print(data_cleaned['hour'].value_counts())
print("----------------------------------------------")
print("Quantidade de eventos por mês: ")
print(data_cleaned['month'].value_counts())
print("----------------------------------------------")

----------------------------------------------
Quantidade de eventos por dia da semana: 
day_of_week
Thursday     3118227
Wednesday    3082678
Tuesday      3034959
Monday       3011976
Friday       2933399
Sunday       2766963
Saturday     2640350
Name: count, dtype: int64
Legenda -> De cima para baixo: Quinta-feira, Quarta-feira, Terça-feira, Segunda-feira, Sexta-feira, Domingo e Sábado
----------------------------------------------
Quantidade de eventos por hora do dia: 
hour
19    1358966
18    1295379
12    1239049
11    1225934
20    1221743
10    1185610
17    1176125
13    1172393
9     1121780
14    1099995
16    1094322
8     1063215
15    1058309
7      963733
21     840801
6      811310
5      601989
22     503133
4      396453
23     291325
3      279344
2      201959
0      201825
1      183860
Name: count, dtype: int64
----------------------------------------------
Quantidade de eventos por mês: 
month
November    4624195
January     4257090
October     4096197
February  

### Estatísticas sobre 'views'

In [95]:
views_event_data = data_cleaned[data_cleaned['event_type'] == 'view']
print(f"Quantidade de eventos de views: {views_event_data.shape[0]}")
print(f"Média de preço de views: {views_event_data['price'].mean()}")
print(f"Preço máximo de views: {views_event_data['price'].max()}")
print(f"Preço mínimo de views: {views_event_data['price'].min()}")
print(f"Média de views por produto: {data_cleaned[data_cleaned['event_type'] == 'view'].groupby('product_id').size().mean()}")
daily_views = data_cleaned[data_cleaned['event_type'] == 'view'].groupby(data_cleaned['event_time'].dt.date).size().reset_index(name='total_views')
print(f"Views diárias:\n {daily_views.head(50)}")
print(f"Média de view por user_id: {data_cleaned[data_cleaned['event_type'] == 'view'].groupby('user_id').size().mean()}")
print(f"Média de view por user_session: {data_cleaned[data_cleaned['event_type'] == 'view'].groupby('user_session').size().mean()}")


Quantidade de eventos de views: 9627329
Média de preço de views: 12.41671085357666
Preço máximo de views: 327.7799987792969
Preço mínimo de views: 0.05000000074505806
Média de views por produto: 180.2735562879185
Views diárias:
     event_time  total_views
0   2019-10-01        61066
1   2019-10-02        76348
2   2019-10-03        56648
3   2019-10-04        53524
4   2019-10-05        49323
5   2019-10-06        72776
6   2019-10-07        75720
7   2019-10-08        64824
8   2019-10-09        65185
9   2019-10-10        60721
10  2019-10-11        56925
11  2019-10-12        52573
12  2019-10-13        53856
13  2019-10-14        62013
14  2019-10-15        59638
15  2019-10-16        62854
16  2019-10-17        61776
17  2019-10-18        57080
18  2019-10-19        49571
19  2019-10-20        52663
20  2019-10-21        63685
21  2019-10-22        62209
22  2019-10-23        61422
23  2019-10-24        59345
24  2019-10-25        52779
25  2019-10-26        51297
26  2019-10-27 

### Estatísticas sobre 'purchase'

In [96]:
purchase_event_data = data_cleaned[data_cleaned['event_type'] == 'purchase']
print(f"Quantidade de eventos de compras: {purchase_event_data.shape[0]}")
print(f"Média de preço de compras: {purchase_event_data['price'].mean()}")
print(f"Preço máximo de compras: {purchase_event_data['price'].max()}")
print(f"Preço mínimo de compras: {purchase_event_data['price'].min()}")
total_purchases_por_produto = purchase_event_data.groupby('product_id').size().reset_index(name='total_purchases')
daily_purchases = purchase_event_data.groupby(purchase_event_data['event_time'].dt.date).size().reset_index(name='total_purchases')
print(f"Compras diárias:\n {daily_purchases.head(50)}")
print(f"Média de compras por user_id: {purchase_event_data.groupby('user_id').size().mean()}")
print(f"Média de compras por user_session: {purchase_event_data.groupby('user_session').size().mean()}")
print(f"Total de compras por produto:\n {total_purchases_por_produto.sort_values(by='total_purchases', ascending=False).head(50)}")


Quantidade de eventos de compras: 1286880
Média de preço de compras: 4.935837745666504
Preço máximo de compras: 327.7799987792969
Preço mínimo de compras: 0.05000000074505806
Compras diárias:
     event_time  total_purchases
0   2019-10-01             8475
1   2019-10-02             9099
2   2019-10-03             8863
3   2019-10-04             7562
4   2019-10-05             5940
5   2019-10-06             7265
6   2019-10-07             9376
7   2019-10-08             8604
8   2019-10-09             8463
9   2019-10-10             8116
10  2019-10-11             6921
11  2019-10-12             6431
12  2019-10-13             7087
13  2019-10-14             8003
14  2019-10-15             8299
15  2019-10-16             9095
16  2019-10-17             9235
17  2019-10-18             7919
18  2019-10-19             6064
19  2019-10-20             6836
20  2019-10-21             8872
21  2019-10-22             8643
22  2019-10-23             8713
23  2019-10-24             8778
24  201

### Taxa de Conversão

In [86]:
# Contar total de eventos por tipo
total_events = data_cleaned.groupby('event_type').size()

# Obter totais para cada tipo de evento
total_views = total_events.get('view', 0)
total_carts = total_events.get('cart', 0)
total_purchases = total_events.get('purchase', 0)
total_remove_from_cart = total_events.get('remove_from_cart', 0)

# Calcular taxas de conversão
rate_conversion_view_to_cart = (total_carts / total_views * 100) if total_views > 0 else 0
rate_conversion_view_to_purchase = (total_purchases / total_views * 100) if total_views > 0 else 0
rate_conversion_cart_to_purchase = (total_purchases / total_carts * 100) if total_carts > 0 else 0
rate_conversion_cart_to_remove = (total_remove_from_cart / total_carts * 100) if total_carts > 0 else 0

# Criar DataFrame com as métricas
rate_conversion_all = pd.DataFrame({
    'Métrica': [
        'Total de Visualizações', 
        'Total de Carrinhos', 
        'Total de Compras', 
        'Total de Remoções do Carrinho', 
        'Taxa de Conversão (Visualizações para Carrinho)', 
        'Taxa de Conversão (Visualizações para Compras)', 
        'Taxa de Conversão (Carrinho para Compras)', 
        'Taxa de Conversão (Carrinho para Remoções)'
    ],
    'Valor': [
        total_views, 
        total_carts, 
        total_purchases, 
        total_remove_from_cart,
        rate_conversion_view_to_cart, 
        rate_conversion_view_to_purchase, 
        rate_conversion_cart_to_purchase,
        rate_conversion_cart_to_remove
    ]
})

# Formatando as taxas de conversão como porcentagens
rate_conversion_all.loc[rate_conversion_all['Métrica'].str.contains('Taxa de Conversão'), 'Valor'] = rate_conversion_all['Valor'].map('{:.2f}%'.format)

# Exibir resultados
print(rate_conversion_all)


C:\Users\icaro\AppData\Local\Temp\ipykernel_8860\1360925227.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_events = data_cleaned.groupby('event_type').size()
C:\Users\icaro\AppData\Local\Temp\ipykernel_8860\1360925227.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['59.40%' '13.37%' '22.50%' '69.16%']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  rate_conversion_all.loc[rate_conversion_all['Métrica'].str.contains('Taxa de Conversão'), 'Valor'] = rate_conversion_all['Valor'].map('{:.2f}%'.format)


                                           Métrica      Valor
0                           Total de Visualizações  9627329.0
1                               Total de Carrinhos  5718963.0
2                                 Total de Compras  1286880.0
3                    Total de Remoções do Carrinho  3955380.0
4  Taxa de Conversão (Visualizações para Carrinho)     59.40%
5   Taxa de Conversão (Visualizações para Compras)     13.37%
6        Taxa de Conversão (Carrinho para Compras)     22.50%
7       Taxa de Conversão (Carrinho para Remoções)     69.16%
